# LCP Project: analysis of MAGIC04 dataset
First, we import the libraries

In [16]:
# math libraries
import numpy as np
import pandas as pd
import numpy.random as npr
import matplotlib.pyplot as plt
import seaborn as sns
import random as rnd
import uncertainties as un

# general OS libraries
from time import time
import gzip
import os
import operator
import warnings
from IPython.core.interactiveshell import InteractiveShell

# ML libraries
import sklearn
from sklearn.metrics import roc_curve, roc_auc_score, auc, confusion_matrix, accuracy_score, make_scorer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.datasets import make_classification
from sklearn.svm import NuSVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.exceptions import ConvergenceWarning
from sklearn.ensemble import RandomForestClassifier

# Part 7: Calculation of the observation time
Assuming that the telescope has a collection area of 10^9 cm^2 and that we are observing gamma rays between 50 GeV and 50 TeV, to what observation time does this measurement corresponds? (assume here the Crab spectrum measured by HEGRA [Aharonian, F. et al. 2000])

------da rivedere descrizione-------
- da capire +- 0.5
- da capire se vogliamo dividere il range 0.05 Tev - 50 Tev in due e usare il primo integrale per 1-50 e il secondo (log) per 0.05-1 (al momento non sono divisi)

To calculate the observation time corresponding to the Cherenkov telescope measurement, we use the Crab Nebula spectrum measured by HEGRA. Then, converting the telescope's collection area to square meters ( $A = 10^9 \, \text{cm}^2 = 10^5 \, \text{m}^2 $), we can determine
the event rate by integrating the flux over the energy range from **50 GeV to 50 TeV**:

$$
R = A \cdot J_{\gamma} =  A \int_{0.05 \, TeV}^{50 \, TeV} \frac{dJ_{\gamma}}{dE} dE
$$

This integration provides the **event rate in photons per second**. Given the total number of recorded events $n_\gamma$, we can then calculate the **observation time** by dividing the number of events by the event rate:

$$
\Delta t = \frac{n_{\gamma}}{R}
$$

In [17]:
# Constants
A = 10**9 * 1e-4  # collection area in m^2
E0 = 1.0  # TeV
Emin = 0.05  # TeV = 50 GeV
Emax = 50  # TeV
n_gamma, sigma_n_gamma =  4199, 0.4 

# Constants for the first integral
alpha1, alpha1_err = 2.59, 0.03
c1, c1_err = 2.79e-7, 0.02e-7

# Constants for the second integral
c2, c2_err = 2.67e-7, 0.01e-7
alpha2, alpha2_err = 2.47, 0.1
beta, beta_err = 0.11, 0.1

Primo integrale con errore dato da derivate parziali

## First integral
According to the study by **Aharonian et al. (2000)**, the differential flux of gamma rays in the range of $1-20 \, \text{TeV}$ is given by:

$$
\frac{dJ_{\gamma}}{dE} = (2.79 \pm 0.02) \times 10^{-7} \left( \frac{E}{1 \, \text{TeV}} \right)^{(-2.59 \pm 0.03)} \, \text{ph} \, \text{m}^{-2} \, \text{s}^{-1} \, \text{TeV}^{-1}
$$
$$
\frac{dJ_{\gamma}}{dE} = c_1 \cdot \left( \frac{E}{E_0} \right)^{- \alpha_1} \, \text{ph} \, \text{m}^{-2} \, \text{s}^{-1} \, \text{TeV}^{-1}
$$
with:
$$
c_1 = (2.79 \pm 0.02) \times 10^{-7}, E_0 = 1 \, \text{TeV}, \alpha_1 = 2.59 \pm 0.03
$$
so the integral will be:
$$
J_\gamma = \frac{E_0^{\alpha_1} \cdot c_1}{-\alpha_1 + 1} \cdot \left( E_{\text{max}}^{-\alpha_1 + 1} - E_{\text{min}}^{-\alpha_1 + 1} \right)
$$
with $E_{\text{min}} = 0.05 \, \text{TeV}, E_{\text{max}} = 50 \, \text{TeV} $

### Error Propagation

We have uncertainties in the following parameters:

$$
c_1 = (2.79 \pm 0.02) \times 10^{-7}
$$
$$
\alpha_1 = 2.59 \pm 0.03
$$

Using the general error propagation formula the total uncertainty in R:

$$
\sigma_R = \sqrt{ \left( \frac{\partial R}{\partial c_1} \sigma_{c_1} \right)^2 + \left( \frac{\partial R}{\partial \alpha_1} \sigma_{\alpha_1} \right)^2 }
$$

The error for the time will be:
$$
\sigma_t = t \sqrt{ \left( \frac{\sigma_{n_{\gamma}}}{n_{\gamma}} \right)^2 + \left( \frac{\sigma_R}{R} \right)^2 }
$$

In [18]:
import sympy as sp

def uncertainty(A, E0, Emin, Emax, c1, c1_err, alpha1, alpha1_err):
    #definisco i simboli
    c1_sym, alpha1_sym, Emin_sym, Emax_sym, E0_sym, A_sym = sp.symbols('c1 alpha1 Emin Emax E0 A')
    
    #definizione di R 
    R = (A_sym * c1_sym * (E0_sym**alpha1_sym) / (-alpha1_sym + 1)) * (Emax_sym**(-alpha1_sym + 1) - Emin_sym**(-alpha1_sym + 1))

    #calcolo le derivate parziali rispetto a c1 e alpha1
    dR_dc1 = sp.diff(R, c1_sym)
    dR_dalpha1 = sp.diff(R, alpha1_sym)
    
    #conversione a funzioni
    dR_dc1_func = sp.lambdify((A_sym, E0_sym, Emin_sym, Emax_sym, c1_sym, alpha1_sym), dR_dc1, 'numpy')
    dR_dalpha1_func = sp.lambdify((A_sym, E0_sym, Emin_sym, Emax_sym, c1_sym, alpha1_sym), dR_dalpha1, 'numpy')
    
    #calcolo il valore delle derivate e di R
    dR_dc1_val = dR_dc1_func(A, E0, Emin, Emax, c1, alpha1)
    dR_dalpha1_val = dR_dalpha1_func(A, E0, Emin, Emax, c1, alpha1)
    R_val = (A * c1 * (E0**alpha1) / (-alpha1 + 1)) * (Emax**(-alpha1 + 1) - Emin**(-alpha1 + 1))
    
    #incertezza totale su R
    sigma_R = np.sqrt((dR_dc1_val * c1_err) ** 2 + (dR_dalpha1_val * alpha1_err) ** 2)
    
    return R_val, sigma_R


#valore dell'errore su R
R_val, sigma_R = uncertainty(A, E0, Emin, Emax, c1, c1_err, alpha1, alpha1_err)
t1 = (n_gamma / R_val) / 60
sigma_t1 = t1 * (np.sqrt((sigma_n_gamma / n_gamma)**2 + (sigma_R/R_val)**2))

print(f"t1 = {t1:.1f} ± {sigma_t1:.1f} minutes")

t1 = 34.1 ± 2.4 minutes


## Second integral
At low energies there is a slight flattening of the spectrum. A fit with a logarithmic steepening of the power law spectrum gives (according to the study by **Aharonian et al. (2000)**):
$$
\frac{dJ_{\gamma}}{dE} = (2.67 \pm 0.01) \times 10^{-7} \left( \frac{E}{1 \, \text{TeV}} \right)^{(-2.47 \pm 0.1) -(0.11 \pm 0.1) \, \log(E)} \, \text{ph} \, \text{m}^{-2} \, \text{s}^{-1} \, \text{TeV}^{-1}
$$
$$
\frac{dJ_{\gamma}}{dE} = c_2 \cdot \left( \frac{E}{E_0} \right)^{- \alpha_2 + \beta \, \log(E)} \, \text{ph} \, \text{m}^{-2} \, \text{s}^{-1} \, \text{TeV}^{-1}
$$
with:
$$
c_2 = (2.67 \pm 0.01) \times 10^{-7}, E_0 = 1 \, \text{TeV}, \alpha_2 = 2.47 \pm 0.1, \beta = -0.11 \pm 0.1
$$
Because of the logarithmic correction, it's impossible to integrate it analytically in a straightforward way. Hence, to calculate this integral, we use two numerical integration methods: Romberg integration and Simpson's Rule.

###  Romberg integration

In [19]:
from scipy.integrate import romb

E_values = np.linspace(Emin, Emax, 2**10+1)
dE = np.abs(E_values[0] - E_values[1])

def dJ_gamma2(E, c2, alpha2, beta):
    return c2 * (E / E0) ** (-alpha2 - beta * np.log(E))

J_gamma_values = dJ_gamma2(E_values, c2, alpha2, beta)
J_gamma = romb(J_gamma_values, dx = dE)

R = A * J_gamma

def partial_derivatives2():
    dJ_dc2 = J_gamma / c2
    dJ_dalpha2 = -J_gamma * np.log(E_values / E0)
    dJ_dalpha2 = romb(dJ_dalpha2, dx = dE)
    dJ_dbeta = -J_gamma * np.log(E_values/E0) * np.log(E_values)
    dJ_dbeta = romb(dJ_dbeta, dx = dE)

    return dJ_dc2, dJ_dalpha2, dJ_dbeta

dJ_dc2, dJ_dalpha2, dJ_dbeta = partial_derivatives2()

sigma_J = np.sqrt(
    (dJ_dc2 * c2_err) ** 2 +
    (dJ_dalpha2 * alpha2_err) ** 2 +
    (dJ_dbeta * beta_err) ** 2
)

sigma_R = sigma_J 

t = (n_gamma / R)/60
sigma_t = t * (np.sqrt((sigma_n_gamma / n_gamma)**2 + (sigma_R/R)**2))
print(f"t = {t:.2f} +- {sigma_t:.2f} minutes")

t = 84.64 +- 0.04 minutes


#### using .quad

In [20]:
from scipy.integrate import quad

#definisco l'integranda
def dJ_gamma(E, c2, alpha2, beta):
    return c2 * (E / E0) ** (-alpha2 - beta * np.log(E))

#definisco R
def R(c2, alpha2, beta):
    integral, _ = quad(dJ_gamma, Emin, Emax, args=(c2, alpha2, beta))
    return A * integral

#Monte Carlo for the error
N = 10000
c2_mc = np.random.normal(c2, c2_err, N)
alpha2_mc = np.random.normal(alpha2, alpha2_err, N)
beta_mc = np.random.normal(beta, beta_err, N)

R_mc = np.array([R(c2, alpha2, beta) for c2, alpha2, beta in zip(c2_mc, alpha2_mc, beta_mc)])

#calcolo valor medio e errore di R
R_mean = np.mean(R_mc)
R_std = np.std(R_mc)/np.sqrt(N)

#propago su t
t2 = (n_gamma / R_mean)/60
sigma_t2 = t2 * (np.sqrt((sigma_n_gamma / n_gamma)**2 + (R_std/R_mean)**2))

print(f"t2 = {t2:.2f} +- {sigma_t2:.2f} minutes")

t2 = 69.91 +- 0.47 minutes


### Simpson's Rule

The trapezoidal rule estimates the area under a curve by approximating the curve with straight-line segments. We can often get a
better result if we approximate the function instead with curves of some kind, Simpson’s rule does exactly this, using quadratic curves.

In [21]:
import numpy as np
from scipy.integrate import simpson

n = int(10e5)
E_values = np.linspace(Emin, Emax, n+1)

def dJ_gamma2(E, c2, alpha2, beta):
    return c2 * (E / E0) ** (-alpha2 - beta * np.log(E))

J_gamma_values = dJ_gamma2(E_values, c2, alpha2, beta)
J_gamma = simpson(J_gamma_values, E_values)

R = A * J_gamma

def partial_derivatives():
    dJ_dc2 = J_gamma / c2
    dJ_dalpha2 = -J_gamma * np.log(E_values / E0)
    dJ_dalpha2 = simpson(dJ_dalpha2, E_values)
    dJ_dbeta = -J_gamma * np.log(E_values/E0) * np.log(E_values)
    dJ_dbeta = simpson(dJ_dbeta, E_values)

    return dJ_dc2, dJ_dalpha2, dJ_dbeta

dJ_dc2, dJ_dalpha2, dJ_dbeta = partial_derivatives()

sigma_J = np.sqrt(
    (dJ_dc2 * c2_err) ** 2 +
    (dJ_dalpha2 * alpha2_err) ** 2 +
    (dJ_dbeta * beta_err) ** 2
)

sigma_R = sigma_J

t2_s = (n_gamma / R)/60
sigma_t2_s = t2_s * (np.sqrt((sigma_n_gamma / n_gamma)**2 + (sigma_R/R)**2))

print(f"t2 = {t2_s:.2f} +- {sigma_t2_s:.2f} minutes")

t2 = 86.18 +- 0.04 minutes


### Monte Carlo

In [23]:
def dJ_gamma2(E, c2, alpha2, beta):
    return c2 * (E / E0) ** (-alpha2 - beta * np.log(E))

n = int(10e5)
E_values = np.linspace(Emin, Emax, n+1)
y_max= np.max(dJ_gamma2(E_values,c2,alpha2,beta)) 


# Monte Carlo integration
N=1000000
count=0
f=[]
for i in range(N):
    x= np.random.uniform(Emin,Emax)
    y= np.random.uniform(0,y_max+10**(-4))
    if y<dJ_gamma2(x, c2, alpha2, beta): count+=1
    f.append(dJ_gamma2(x, c2, alpha2, beta))
I=(Emax-Emin)*(y_max+10**(-4))*count/N
sigma_f = np.std(f)
sigma_I= (Emax-Emin)*sigma_f/N**(0.5)
T=n_gamma/(A*I)/60
sigma=T*np.sqrt((sigma_n_gamma/n_gamma)**2+(sigma_I/I)**2)
print(T, sigma)

88.16204768668301 1.690997260143069


# Part 8: Do the same using the MAGIC measured spectrum of the Crab Nebula [Aleksic, J. et al. 2015]

In [24]:
# Constants
A = 10**9 * 1e-4  # collection area in m^2
E0 = 1.0  # TeV
Emin = 0.05  # TeV = 50 GeV
Emax = 50  # TeV
n_gamma, sigma_n_gamma = 4199, 0.4 

# Constants for the first integral
f0, f0_err = 3.80e-11, 0.11e-11
alpha, alpha_err = 2.21, 0.02
Ec, Ec_err = 6.0, 0.6

# Constants for the second integral
f02, f02_err = 3.23e-11, 0.03e-11
alpha2, alpha2_err = 2.47, 0.01
beta2, beta2_err = -0.24, 0.01

power law with an exponential cut off:
$$
\frac{dJ_{\gamma}}{dE} = f_0 \cdot \left( \frac{E}{E_0} \right)^{- \alpha} e^{-E/E_C} \, \text{ph} \, \text{m}^{-2} \, \text{s}^{-1} \, \text{TeV}^{-1}
$$
$$
f_0 = (3.80 \pm 0.11) \times 10^{-11} \, \text{TeV}^{-1} \, \text{cm}^{-2} \, \text{s}^{-1}, \alpha = 2.21 \pm 0.02, E_C = (6.0 \pm 0.6) \, \text{TeV}, E_0 = 1 \, \text{TeV}
$$

In [25]:
def dJ_gamma(E, f0, alpha, Ec):
    return f0 * (E / E0) ** (-alpha) * np.exp(-E / Ec)

n = int(10e5)
E_values = np.linspace(Emin, Emax, n+1)
y_max= np.max(dJ_gamma(E_values, f0, alpha, Ec)) 


# Monte Carlo integration
N=100000
count=0
f=[]
for i in range(N):
    x= np.random.uniform(Emin,Emax)
    y= np.random.uniform(0,y_max+10**(-4))
    if y < dJ_gamma(x, f0, alpha, Ec): count+=1
    f.append(dJ_gamma(x, f0, alpha, Ec))
I=(Emax-Emin)*(y_max+10**(-4))*count/N
sigma_f = np.std(f)
sigma_I= (Emax-Emin)*sigma_f/N**(0.5)
t=n_gamma/(A*I)/60
sigma=t*np.sqrt((sigma_n_gamma/n_gamma)**2+(sigma_I/I)**2)
print(t, sigma)

inf inf


C:\Users\sofia\AppData\Local\Temp\ipykernel_27024\677154961.py:21: RuntimeWarning: divide by zero encountered in scalar divide
  t=n_gamma/(A*I)/60
C:\Users\sofia\AppData\Local\Temp\ipykernel_27024\677154961.py:22: RuntimeWarning: divide by zero encountered in scalar divide
  sigma=t*np.sqrt((sigma_n_gamma/n_gamma)**2+(sigma_I/I)**2)


## Second integral (Log-parabola)

$$
\frac{dJ_{\gamma}}{dE} = f_0 \cdot \left( \frac{E}{E_0} \right)^{- \alpha + \beta \, \log(E / E_0)} \, \text{ph} \, \text{m}^{-2} \, \text{s}^{-1} \, \text{TeV}^{-1}
$$
$$
f_0 = (3.23 \pm 0.03) \times 10^{-11} \, \text{TeV}^{-1} \, \text{cm}^{-2} \, \text{s}^{-1}, \alpha = 2.47 \pm 0.01, \beta = -0.24 \pm 0.01, E_0 = 1 \, \text{TeV}
$$

### Simpson's Rule

The trapezoidal rule estimates the area under a curve by approximating the curve with straight-line segments. We can often get a
better result if we approximate the function instead with curves of some kind, Simpson’s rule does exactly this, using quadratic curves.